In [ ]:
import warnings

from pathlib import Path
from dotenv import load_dotenv
from functools import lru_cache
from IPython.display import display, Markdown

from persuasion_bias.config.logging import SimpleLogger
from persuasion_bias.rag.chains import CompressionRAGChain


warnings.filterwarnings("ignore")
_ = load_dotenv(override=True)

config_file = Path("persuasion_bias") / "config" / "log-config.yaml"
logger = SimpleLogger(name=__name__)
logger.reconfigure_from_file(config_file=config_file)
log = logger.logger

In [3]:
compressionRAG = CompressionRAGChain(log=log)

[ERROR]: You must specify either `repo_id` or set `use_anthropic`.


[STDERR: ERROR 2025-09-11 13:49:22,603 chains.py:168]


ValueError: You must specify either `repo_id` or set `use_anthropic`.

In [4]:
compressionRAG = CompressionRAGChain(log=log, use_anthropic=True)

log.info(compressionRAG.__doc__)

[INFO]: Using Anthropic model: claude-3-5-haiku-20241022
[INFO]: 
    +---------------------------------------------------------------+
    | One-turn RAG runnable with contextual compression retrieval.  |
    | Optionally selects:                                           |
    |       - Claude 3.5 Haiku                                      |
    |       - similarity_threshold for relevance filtering          |
    +---------------------------------------------------------------+
    


In [5]:
compression_retriever = compressionRAG._get_contextual_compression_retriever(similarity_threshold=0.45)
compressionRAG._retriever = compression_retriever

runnable_seq = compressionRAG.create_runnable_sequence()

[INFO]: Creating contextual compression retriever
[INFO]: Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
[INFO]: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[INFO]: Retriever created and cached


In [6]:
@lru_cache(maxsize=16)
def argue(argument: str) -> None:
    """Invoke runnable sequence -  Markdown display"""

    log.info("Invoking the model\n")
    try:
        response = runnable_seq.invoke(argument)
        display(Markdown(response))
    except Exception as e:
        log.exception(e)

In [7]:
argue(argument="Social media is harmful for the youth.")

[INFO]: Invoking the model

[INFO]: HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


## Argument Analysis: Social Media's Impact on Youth

### Bias Detection
- **Bias Score**: 7/10 (High potential for bias)
- **Persuasiveness**: Moderate

### Key Arguments Against Social Media for Youth

1. **Psychological Impact**
   - Increases anxiety and depression
   - Promotes unrealistic social comparisons
   - Damages self-esteem

2. **Developmental Concerns**
   - Disrupts natural social skill development
   - Reduces face-to-face interaction
   - Potentially addictive technology

3. **Evidence-Based Risks
   - Studies show correlation with:
     * Mental health issues
     * Decreased attention spans
     * Reduced academic performance

### Counterarguments

1. Digital literacy is crucial
2. Social connections can be beneficial
3. Moderation and parental guidance are key

### Recommendation
- Implement structured, supervised digital engagement
- Focus on education about responsible technology use
- Prioritize mental health monitoring

### Conclusion
While risks exist, a nuanced approach is more effective than outright banning.